In [ ]:
'''수입원장에서 피스토스 수금수수료 명세를 추출합니다 '''
import pandas as pd
import GladUtil as gd

ym  = '202401'
# dir0  = f'E:/글로벌백업(180329)/04.수수료작업/{ym}수수료/'
dir0 = f'd:/글로벌백업(180329)/04.수수료작업/{ym}수수료/'
         
file0 = f'{ym}_수입원장.xlsx'  #원본
brh = '글래드피스토스'  #지점
ins = '장기'           #보종
fc_name = ["고경숙","홍미숙"]

ff = dir0+file0
df = pd.read_excel(ff, sheet_name='피스토스수금', header=3, usecols='a:l').iloc[:13]

In [ ]:
col = ['보험사','증권번호','계약종류','지점','사원번호','사원명','FC코드','계약일자','계약자명']
df_all = pd.DataFrame()
for idx, row in df[['보험사','수금 값열','수금값 제목']].iterrows():
    cols = col + [ row['수금값 제목'] ]
    if row['보험사'] !='계':
        df_temp = (pd.read_excel(ff,sheet_name=row['보험사'], usecols=cols, dtype={'증권번호': 'object','계약일자': 'object'})
                  .rename(columns={row['수금값 제목']:'총괄수금수수료'}))
        if brh:
            df_temp = df_temp[df_temp['지점']==brh]
        if ins:    
            df_temp = df_temp[df_temp['계약종류']==ins]
        df_all = pd.concat([df_all, df_temp])
df_all['수금수수료']= df_all['총괄수금수수료'] * .95

# 피스토스 수금은 95%값
df_wrt1 = df_all.drop(columns='총괄수금수수료')
# 사원수금은 100%값을 추출함 : 정산 오류가 있었음
df_wrt2 = (df_all.query("계약일자 >= '20200801'  and  사원명 in (@fc_name) ")
            .drop(columns='수금수수료')
            .rename(columns={'총괄수금수수료':'수금수수료'}))

file0 = f'{ym}_수금수수료명세(피스토스).xlsx'
with pd.ExcelWriter(dir0+file0,mode="w",engine='openpyxl') as writer:  
    df_wrt1.to_excel(writer, sheet_name='피스토스',index=False)
    df_wrt2.to_excel(writer, sheet_name='사원(고경숙,홍미숙)',index=False)